In [1]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
from datetime import datetime,timedelta
from sklearn import preprocessing

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 6, 6

#中文字体
import matplotlib  
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False  

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
full_data = pd.read_csv('./input/full_data_mode.csv')
full_data = full_data.drop(['Unnamed: 0'], axis=1)

full_data.head()

MemoryError: 

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

train.shape
train.head()
test.shape
test.head()

(7377418, 6)

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


(2556790, 6)

,id,msno,song_id,source_system_tab,source_screen_name,source_type
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,song-based-playlist
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio


In [8]:
from scipy.sparse import csr_matrix

In [3]:
train['id'] = -1

train_spa = train[['id', 'msno', 'song_id', 'target']]
test_spa = test[['id', 'msno', 'song_id']]

del train
del test

In [4]:
train_spa.shape
train_spa.head()
test_spa.shape
test_spa.head()

(7377418, 4)

,id,msno,song_id,target
0,-1,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1
1,-1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1
2,-1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1
3,-1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1
4,-1,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1


(2556790, 3)

,id,msno,song_id
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=


In [82]:
len(set(test_spa.msno.unique().tolist()) & set(train_spa.msno.unique().tolist()))

21483

In [ ]:
a = []
for x in test_spa.msno:
    if x not in train_spa.msno:
        a.append(x)
a

KeyboardInterrupt: 

In [62]:
del a

In [5]:
train_spa.target.value_counts()

1    3714656
0    3662762
Name: target, dtype: int64

In [6]:
train_spa.target[train_spa.target == 0] = -1
train_spa.target.value_counts()

test_spa['target'] = 0

 1    3714656
-1    3662762
Name: target, dtype: int64

In [65]:
msno_le = preprocessing.LabelEncoder().fit(train_spa['msno'])
train_spa['msno'] = msno_le.transform(train_spa['msno'])

songid_le = preprocessing.LabelEncoder().fit(train_spa['song_id'])
train_spa['song_id'] = songid_le.transform(train_spa['song_id'])

train_spa.info()
train_spa.as_matrix()

train_spa.to_csv('./model/spa_train.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7377418 entries, 0 to 7377417
Data columns (total 3 columns):
msno       int64
song_id    int64
target     int64
dtypes: int64(3)
memory usage: 168.9 MB


array([[  8158,  74679,      1],
       [ 17259, 223479,      1],
       [ 17259, 120758,      1],
       ..., 
       [ 18266,  51546,      1],
       [  1216, 102430,      1],
       [  1216, 123573,      1]])

In [66]:
msno_le = preprocessing.LabelEncoder().fit(test_spa['msno'])
test_spa['msno'] = msno_le.transform(test_spa['msno'])

songid_le = preprocessing.LabelEncoder().fit(test_spa['song_id'])
test_spa['song_id'] = songid_le.transform(test_spa['song_id'])

test_spa.info()
test_spa.as_matrix()

test_spa.to_csv('./model/spa_test.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556790 entries, 0 to 2556789
Data columns (total 3 columns):
msno       int64
song_id    int64
target     int64
dtypes: int64(3)
memory usage: 58.5 MB


array([[ 12934, 122191,      0],
       [ 12934, 217907,      0],
       [   712,  37385,      0],
       ..., 
       [ 18800, 219419,      0],
       [ 18800, 121326,      0],
       [ 18800, 154308,      0]])

In [7]:
full_spa = pd.concat([train_spa, test_spa])
full_spa.shape
full_spa.head()
full_spa.target.value_counts()

(9934208, 4)

,id,msno,song_id,target
0,-1,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1
1,-1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1
2,-1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1
3,-1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1
4,-1,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1


 1    3714656
-1    3662762
 0    2556790
Name: target, dtype: int64

In [53]:
del train_spa
del test_spa

In [8]:
msno_le = preprocessing.LabelEncoder().fit(full_spa['msno'])
full_spa['msno'] = msno_le.transform(full_spa['msno'])

songid_le = preprocessing.LabelEncoder().fit(full_spa['song_id'])
full_spa['song_id'] = songid_le.transform(full_spa['song_id'])

full_spa.info()
full_spa.shape
full_spa.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9934208 entries, 0 to 2556789
Data columns (total 4 columns):
id         int64
msno       int64
song_id    int64
target     int64
dtypes: int64(4)
memory usage: 379.0 MB


(9934208, 4)

,id,msno,song_id,target
0,-1,9176,86884,1
1,-1,19273,260594,1
2,-1,19273,140755,1
3,-1,19273,27577,1
4,-1,9176,38706,1


In [9]:
full_spa.to_csv('./model/spa_full.csv', index=False)

In [10]:
train_spa = full_spa[full_spa.id == -1]
test_spa = full_spa[full_spa.id != -1]

train_spa.shape
train_spa.head()
test_spa.shape
test_spa.head()

(7377418, 4)

,id,msno,song_id,target
0,-1,9176,86884,1
1,-1,19273,260594,1
2,-1,19273,140755,1
3,-1,19273,27577,1
4,-1,9176,38706,1


(2556790, 4)

,id,msno,song_id,target
0,0,17724,228222,0
1,1,17724,407039,0
2,2,977,70211,0
3,3,1878,419092,0
4,4,1878,160096,0


In [11]:
del full_spa

In [29]:
len(train_spa.msno.value_counts())
len(test_spa.msno.value_counts())
len(train_spa.song_id.value_counts())

30755

25131

359966

In [30]:
from sklearn.cross_validation import KFold

In [36]:
kf = KFold(train_spa.shape[0], n_folds=3, random_state=0)

for i, (train_index, val_index) in enumerate(kf):
        train_spa_kflod = train_spa.iloc[train_index]
        val_spa_kflod = train_spa.iloc[val_index]
        break
        
train_spa_kflod.shape
train_spa_kflod.head()
val_spa_kflod.shape
val_spa_kflod.head()

(4918278, 4)

,id,msno,song_id,target
2459140,-1,943,116542,-1
2459141,-1,9353,74580,1
2459142,-1,15938,381187,1
2459143,-1,943,333634,1
2459144,-1,28614,151746,-1


(2459140, 4)

,id,msno,song_id,target
0,-1,9176,86884,1
1,-1,19273,260594,1
2,-1,19273,140755,1
3,-1,19273,27577,1
4,-1,9176,38706,1


In [37]:
train_spa_kflod.to_csv('./model/spa_train_kflod.csv', index=False)
val_spa_kflod.to_csv('./model/spa_val_kflod.csv', index=False)
test_spa.to_csv('./model/spa_test.csv', index=False)